#### SQLite port_lite database: sales, stocks table
#### csv files: daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 1, 8)

### Yesterday = Last closed business day

In [2]:
yesterday = today - timedelta(days=2)
yesterday

datetime.date(2022, 1, 7)

### Restart and Run All Cells

In [3]:
format_dict = {'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
               'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
               'fm_price':'{:,.2f}','to_price':'{:,.2f}','diff':'{:,.2f}',
               'max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}',
               'volume':'{:,.2f}','beta':'{:,.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}',
               'pct':'{:,.2f}%','percent':'{:,.2f}%'
              }

In [4]:
sql = '''
SELECT * FROM sales ORDER BY id DESC LIMIT 1
'''
tmp = pd.read_sql(sql, conlite)
tmp['fm_date'] = pd.to_datetime(tmp['fm_date'])
tmp['to_date'] = pd.to_datetime(tmp['to_date'])
tmp['created_at'] = pd.to_datetime(tmp['created_at'])
tmp['updated_at'] = pd.to_datetime(tmp['updated_at'])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1085683,WHART,2021-12-28,2022-01-07,8,12.80,12.20,-0.60,-4.69%,-6,-1,12.80,12.10,"2,555,508",2022-01-09,2022-01-09,1


In [5]:
names = tmp['name']
name = names.to_string(index=False)
sql = '''
SELECT * FROM stocks
WHERE name = '%s'
'''
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.68,I,0.00,12.80,17.87,0.41,12.50,"9,000",-4,4,"10,000",0,0,DOS,SET


In [6]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)/fm_price*100,2) AS percent,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "%s" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)

df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.head().style.format(format_dict)


SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)/fm_price*100,2) AS percent,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "2022-01-07" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



,fm_date,to_date,fm_price,to_price,percent,qty,max_price,min_price,status,market
name,,,,,,,,,,
IP,2022-01-05,2022-01-07,19.20,19.30,0.52%,"800,830",19.40,19.00,B,mai
KCE,2022-01-04,2022-01-07,88.50,84.00,-5.08%,"10,261,285",86.25,83.50,B,SET50
NOBLE,2022-01-04,2022-01-07,6.25,6.20,-0.80%,"1,520,836",6.25,6.15,B,SET
PTT,2022-01-06,2022-01-07,38.00,38.75,1.97%,"59,559,294",38.75,38.25,B,SET50
RATCH,2022-01-04,2022-01-07,45.25,44.25,-2.21%,"3,484,963",45.25,44.00,B,SET50


### Price change up or down more than 3%

In [7]:
#df['bdays'] = np.busday_count(df['fm_date'].dt.date,df['to_date'].dt.date)
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
IMH,34.43%,I,12.20,16.40,mai,2022-01-04,3,16.80,12.30
TU,5.13%,I,19.50,20.50,SET50,2021-12-30,8,20.70,19.60
TVO,4.17%,O,30.00,31.25,SET,2021-12-27,11,31.25,30.00
BGRIM,-3.09%,I,40.50,39.25,SET50,2021-12-30,8,40.75,39.00
WHART,-4.69%,I,12.80,12.20,SET,2021-12-28,10,12.80,12.10
KCE,-5.08%,B,88.50,84.00,SET50,2022-01-04,3,86.25,83.50
SYNEX,-11.64%,I,36.50,32.25,SET100,2021-12-30,8,35.75,31.50


In [8]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

watch_list.sort_values(['percent'],ascending=[True]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[True]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[True]).to_csv(box_file)

### Create daily-sales from sales

In [9]:
df.head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,percent,qty,max_price,min_price,status,market,bdays
name,,,,,,,,,,,
IP,2022-01-05,2022-01-07,19.20,19.30,0.52%,"800,830",19.40,19.00,B,mai,2
KCE,2022-01-04,2022-01-07,88.50,84.00,-5.08%,"10,261,285",86.25,83.50,B,SET50,3
NOBLE,2022-01-04,2022-01-07,6.25,6.20,-0.80%,"1,520,836",6.25,6.15,B,SET,3
PTT,2022-01-06,2022-01-07,38.00,38.75,1.97%,"59,559,294",38.75,38.25,B,SET50,1
RATCH,2022-01-04,2022-01-07,45.25,44.25,-2.21%,"3,484,963",45.25,44.00,B,SET50,3


In [10]:
file_percent = "daily-sales.csv"
data_file = data_path + file_percent
output_file = csv_path + file_percent
box_file = box_path + file_percent

df.sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df.sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df.sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)

### Create monitors from stocks

In [11]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.head()

,status
name,
BCH,I
BGC,O
BGRIM,I
EPG,I
GLOBAL,I


In [12]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
WHERE status IN ("B","I","O","S")
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,9
1,I,15
2,O,4


### output_file = iCloud Drive on PC

In [13]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)

### End of daily process

In [14]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,9
1,I,15
2,O,4
3,T,5
4,U,5
5,X,12


In [15]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
IP,B,mai
KCE,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
STA,B,SET50
TISCO,B,SET50
TSTH,B,SET
TYCN,B,SET


In [16]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [17]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BGC,O
IP,B
KCE,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O
STA,B
TCAP,O


In [18]:
buy_candidates.shape[0]

13

In [19]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BCH,I
BGRIM,I
EPG,I
GLOBAL,I
IMH,I
IVL,I
KBANK,I
NER,I
RJH,I


In [20]:
sell_candidates.shape[0]

15